In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import numpy as np
import copy
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else "cpu"
print(f'Device is {device}')

Device is cuda:0


In [3]:
data_transforms = {
 'train': transforms.Compose([
 transforms.Resize((224,224)),
 transforms.ToTensor(),
 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
 ]),
 'val': transforms.Compose([
 transforms.Resize((224,224)),
 transforms.ToTensor(),
 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
 ]),
 'test': transforms.Compose([
 transforms.Resize((224,224)),
 transforms.ToTensor(),
 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
 ]),
}

In [5]:
# unzip the  file
import zipfile
with zipfile.ZipFile("hw2_data.zip","r") as zip_ref:
    zip_ref.extractall("hw2_data")

In [6]:
data_dir = '/content/hw2_data/hw2_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
 for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8, shuffle=True, num_workers=4)
 for x in ['train', 'val' , 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs ))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(best_model_wts ,'best_model_weight.pth')

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [8]:
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features

model.classifier[6] = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [9]:
model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Epoch 1/25
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 1.5683 Acc: 0.5386
val Loss: 1.3682 Acc: 0.5861

Epoch 2/25
----------
train Loss: 1.1253 Acc: 0.6504
val Loss: 1.2627 Acc: 0.6282

Epoch 3/25
----------
train Loss: 0.9324 Acc: 0.7004
val Loss: 1.2981 Acc: 0.6071

Epoch 4/25
----------
train Loss: 0.8018 Acc: 0.7355
val Loss: 1.3278 Acc: 0.5840

Epoch 5/25
----------
train Loss: 0.7060 Acc: 0.7583
val Loss: 1.2087 Acc: 0.6492

Epoch 6/25
----------
train Loss: 0.6343 Acc: 0.7780
val Loss: 1.3393 Acc: 0.6324

Epoch 7/25
----------
train Loss: 0.5630 Acc: 0.7882
val Loss: 1.2694 Acc: 0.6450

Epoch 8/25
----------
train Loss: 0.4144 Acc: 0.8259
val Loss: 1.3065 Acc: 0.6555

Epoch 9/25
----------
train Loss: 0.3673 Acc: 0.8342
val Loss: 1.3339 Acc: 0.6534

Epoch 10/25
----------
train Loss: 0.3573 Acc: 0.8272
val Loss: 1.3445 Acc: 0.6618

Epoch 11/25
----------
train Loss: 0.3378 Acc: 0.8319
val Loss: 1.3693 Acc: 0.6639

Epoch 12/25
----------
train Loss: 0.3432 Acc: 0.8288
val Loss: 1.4003 Acc: 0.6555

Epoch 13/25
----------


In [10]:

print('Testing')
model = models.vgg16()
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 20)
model = model.to(device)
model.load_state_dict(torch.load('/content/best_model_weight.pth'))


Testing


<All keys matched successfully>

In [11]:
def get_test_results(model, phase):

	model.eval()
	
	y_true = []
	y_pred = []
	
	for inputs, labels in dataloaders[phase]:
		inputs = inputs.to(device)
		outputs = model(inputs)
		_, preds = torch.max(outputs, 1)
		y_true.extend(labels)
		y_pred.extend(preds.to('cpu'))
		
	return y_true, y_pred

In [12]:
y_true, y_pred = get_test_results(model, 'train')
print(f"Accuracy Score: {accuracy_score(y_true, y_pred)}\n")
print(f"Confusion Matrix\n")
print(np.array2string(confusion_matrix(y_true, y_pred)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Accuracy Score: 0.8449855983241686

Confusion Matrix

[[194   0   0   2   0   0   4   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0 182   0   1   2   5   1   0   0   1   0   0   0   2   1   2   0   0
    2   1]
 [  0   0 195   0   0   0   1   0   0   0   0   1   0   0   1   1   0   0
    0   1]
 [  0   0   1 196   0   0   0   0   0   0   0   0   0   1   1   0   0   1
    0   0]
 [  0   5   0   0 149   0   1   2   4   0  16   3   1   0   3   1   1   2
    0  12]
 [  0   2   0   0   0 161   6   0   0   0   0   0   0   2   0   0   0   0
    0   0]
 [  2   9   0   3   2  12 156   1   0   0   0   1   0   4   7   1   0   0
    2   0]
 [  0   0   0   0   3   0   1 181   3   0   1   5   0   0   0   1   0   2
    0   3]
 [  0   1   0   0   6   0   1   3  96   0  27   6   0   0   5  17   0   8
    0  30]
 [  0   0   0   0   0   0   1   0   0 120   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0  15   0   0   0   9   0 153   1   0   0   1  14   0   4
    0   3]
 [  0   0  


# Variation 1 

In [16]:
#Learning rate = 0.001
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features

model.classifier[6] = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

#Training
model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Epoch 1/25
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 1.6840 Acc: 0.5135
val Loss: 1.1981 Acc: 0.6387

Epoch 2/25
----------
train Loss: 1.0901 Acc: 0.6646
val Loss: 1.1311 Acc: 0.6576

Epoch 3/25
----------
train Loss: 0.9251 Acc: 0.7057
val Loss: 1.1304 Acc: 0.6639

Epoch 4/25
----------
train Loss: 0.8199 Acc: 0.7261
val Loss: 1.1313 Acc: 0.6576

Epoch 5/25
----------
train Loss: 0.7410 Acc: 0.7549
val Loss: 1.0975 Acc: 0.6555

Epoch 6/25
----------
train Loss: 0.6921 Acc: 0.7578
val Loss: 1.1647 Acc: 0.6576

Epoch 7/25
----------
train Loss: 0.6277 Acc: 0.7748
val Loss: 1.1557 Acc: 0.6471

Epoch 8/25
----------
train Loss: 0.5295 Acc: 0.7989
val Loss: 1.1967 Acc: 0.6513

Epoch 9/25
----------
train Loss: 0.4990 Acc: 0.8089
val Loss: 1.2056 Acc: 0.6513

Epoch 10/25
----------
train Loss: 0.4814 Acc: 0.8149
val Loss: 1.2062 Acc: 0.6513

Epoch 11/25
----------
train Loss: 0.4853 Acc: 0.8115
val Loss: 1.2180 Acc: 0.6555

Epoch 12/25
----------
train Loss: 0.4690 Acc: 0.8143
val Loss: 1.2192 Acc: 0.6618

Epoch 13/25
----------


In [17]:
print('Testing')
model = models.vgg16()
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 20)
model = model.to(device)
model.load_state_dict(torch.load('/content/best_model_weight.pth'))

Testing


<All keys matched successfully>

In [18]:
y_true, y_pred = get_test_results(model, 'train')
print(f"Accuracy Score: {accuracy_score(y_true, y_pred)}\n")
print(f"Confusion Matrix\n")
print(np.array2string(confusion_matrix(y_true, y_pred)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Accuracy Score: 0.8394867766431003

Confusion Matrix

[[197   0   0   1   0   1   1   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0 181   0   0   2   4   4   0   0   1   0   0   0   4   1   1   0   0
    1   1]
 [  0   0 196   1   0   0   0   0   0   0   0   1   0   0   1   0   1   0
    0   0]
 [  1   1   0 193   0   0   1   0   0   0   0   0   0   1   2   0   0   1
    0   0]
 [  0   5   0   0 126   0   2   3   5   0  23   5   2   0   4   1   1   8
    0  15]
 [  0   3   0   0   0 161   3   0   0   0   0   0   0   2   1   0   0   0
    1   0]
 [  5   4   1   3   2  16 148   0   0   1   0   4   0   5   4   3   0   0
    4   0]
 [  0   0   0   0   3   0   1 181   1   0   0   5   0   0   0   1   1   4
    0   3]
 [  0   1   0   0   6   0   2   5  92   0  28   4   0   0   8  16   0  13
    1  24]
 [  0   0   0   0   0   0   0   0   0 120   0   0   0   0   0   0   1   0
    0   0]
 [  0   0   0   0   9   0   0   2   9   0 161   1   0   0   1   7   0   9
    0   1]
 [  0   0  

# Variation 2

In [20]:
#Learning rate = 0.002
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features

model.classifier[6] = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

#Training
model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Epoch 1/25
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 1.7671 Acc: 0.4716
val Loss: 1.4304 Acc: 0.5546

Epoch 2/25
----------
train Loss: 1.3259 Acc: 0.5855
val Loss: 1.4939 Acc: 0.5399

Epoch 3/25
----------
train Loss: 1.0910 Acc: 0.6502
val Loss: 1.4645 Acc: 0.5777

Epoch 4/25
----------
train Loss: 0.9484 Acc: 0.6884
val Loss: 1.3951 Acc: 0.5819

Epoch 5/25
----------
train Loss: 0.8104 Acc: 0.7287
val Loss: 1.5656 Acc: 0.5693

Epoch 6/25
----------
train Loss: 0.7043 Acc: 0.7612
val Loss: 1.3578 Acc: 0.6345

Epoch 7/25
----------
train Loss: 0.6362 Acc: 0.7795
val Loss: 1.4823 Acc: 0.5609

Epoch 8/25
----------
train Loss: 0.4516 Acc: 0.8267
val Loss: 1.3729 Acc: 0.6218

Epoch 9/25
----------
train Loss: 0.3958 Acc: 0.8225
val Loss: 1.4197 Acc: 0.6303

Epoch 10/25
----------
train Loss: 0.3611 Acc: 0.8322
val Loss: 1.4186 Acc: 0.6303

Epoch 11/25
----------
train Loss: 0.3554 Acc: 0.8277
val Loss: 1.4768 Acc: 0.6303

Epoch 12/25
----------
train Loss: 0.3443 Acc: 0.8251
val Loss: 1.4811 Acc: 0.6303

Epoch 13/25
----------


In [21]:
print('Testing')
model = models.vgg16()
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 20)
model = model.to(device)
model.load_state_dict(torch.load('/content/best_model_weight.pth'))

Testing


<All keys matched successfully>

In [22]:
y_true, y_pred = get_test_results(model, 'train')
print(f"Accuracy Score: {accuracy_score(y_true, y_pred)}\n")
print(f"Confusion Matrix\n")
print(np.array2string(confusion_matrix(y_true, y_pred)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Accuracy Score: 0.8447237496726892

Confusion Matrix

[[197   0   0   1   0   0   2   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0 182   0   0   4   4   3   0   0   0   0   0   0   1   1   2   0   0
    2   1]
 [  0   0 193   0   0   0   1   0   1   0   0   1   0   0   1   1   1   0
    0   1]
 [  1   0   1 192   0   0   1   0   0   0   0   0   0   0   4   0   0   1
    0   0]
 [  0   3   0   0 140   0   2   1   4   0  20   5   2   0   2   3   0   4
    0  14]
 [  0   3   0   0   0 161   4   0   0   0   0   0   0   2   0   0   0   0
    1   0]
 [  4   7   0   3   1  14 152   0   0   0   0   3   0   7   4   2   0   0
    3   0]
 [  0   0   0   0   3   0   1 184   1   0   1   4   0   0   0   1   1   2
    0   2]
 [  0   1   0   0   6   0   1   5 108   0  32   5   0   0   3   5   0  12
    1  21]
 [  0   1   0   0   0   0   1   0   0 118   0   0   0   0   0   0   1   0
    0   0]
 [  0   0   0   0   8   0   0   0   9   0 158   1   0   0   1  13   0   8
    0   2]
 [  0   0  

# Variation 3

In [27]:
#Optimizer : Adadelta algorithm
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features

model.classifier[6] = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.Adadelta(model.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [28]:

#Training
model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Epoch 1/25
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 2.9110 Acc: 0.1026
val Loss: 2.7174 Acc: 0.2227

Epoch 2/25
----------
train Loss: 2.5485 Acc: 0.2257
val Loss: 2.1467 Acc: 0.3950

Epoch 3/25
----------
train Loss: 2.0809 Acc: 0.3700
val Loss: 1.7533 Acc: 0.4874

Epoch 4/25
----------
train Loss: 1.8084 Acc: 0.4454
val Loss: 1.5509 Acc: 0.5483

Epoch 5/25
----------
train Loss: 1.6017 Acc: 0.5109
val Loss: 1.4271 Acc: 0.5714

Epoch 6/25
----------
train Loss: 1.4779 Acc: 0.5452
val Loss: 1.3610 Acc: 0.6029

Epoch 7/25
----------
train Loss: 1.3787 Acc: 0.5789
val Loss: 1.3108 Acc: 0.5987

Epoch 8/25
----------
train Loss: 1.2948 Acc: 0.5996
val Loss: 1.3069 Acc: 0.6029

Epoch 9/25
----------
train Loss: 1.2811 Acc: 0.6070
val Loss: 1.3014 Acc: 0.6050

Epoch 10/25
----------
train Loss: 1.3068 Acc: 0.6083
val Loss: 1.2998 Acc: 0.6092

Epoch 11/25
----------
train Loss: 1.2731 Acc: 0.6096
val Loss: 1.2959 Acc: 0.6134

Epoch 12/25
----------
train Loss: 1.2760 Acc: 0.6132
val Loss: 1.2916 Acc: 0.6134

Epoch 13/25
----------


In [29]:
print('Testing')
model = models.vgg16()
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 20)
model = model.to(device)
model.load_state_dict(torch.load('/content/best_model_weight.pth'))

Testing


<All keys matched successfully>

In [30]:
y_true, y_pred = get_test_results(model, 'train')
print(f"Accuracy Score: {accuracy_score(y_true, y_pred)}\n")
print(f"Confusion Matrix\n")
print(np.array2string(confusion_matrix(y_true, y_pred)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Accuracy Score: 0.7054202670856246

Confusion Matrix

[[189   0   1   4   0   0   4   0   1   0   0   0   0   1   0   0   0   0
    0   0]
 [  0 160   2   4   1   3   5   0   0   0   1   0   2  14   1   1   0   0
    6   0]
 [  0   0 183   3   3   0   1   3   2   0   0   0   0   0   1   1   3   0
    0   0]
 [  3   1   1 182   0   0   5   0   0   0   0   1   0   1   1   0   1   2
    2   0]
 [  0   8   3   3  67   0   2   6   6   0  38   7   1   0  27   2   0   8
    2  20]
 [  0   3   0   2   0 148  13   0   0   0   0   0   0   1   0   0   0   0
    4   0]
 [  6   7   0   8   1  20 127   0   1   0   0   5   0   7   7   2   1   0
    7   1]
 [  0   0   4   0   1   0   0 170   0   0   2  11   0   0   0   0   4   7
    0   1]
 [  0   3   0   2   8   0   3   6  50   0  31   4   0   0  16   8   0  36
    0  33]
 [  0   0   4   1   1   0   2   1   0  73   0   4  18   0   2   0  15   0
    0   0]
 [  0   0   1   0  10   0   0   1  26   0 133   1   0   0   5   4   0  14
    0   5]
 [  0   0  

# Variation 4

In [32]:
# Optimizer : Adam
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features

model.classifier[6] = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

#Training
model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

Epoch 1/25
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 3.0333 Acc: 0.0542
val Loss: 2.9978 Acc: 0.0525

Epoch 2/25
----------
train Loss: 3.0047 Acc: 0.0490
val Loss: 2.9995 Acc: 0.0525

Epoch 3/25
----------
train Loss: 3.0003 Acc: 0.0450
val Loss: 3.0013 Acc: 0.0525

Epoch 4/25
----------
train Loss: 3.0014 Acc: 0.0458
val Loss: 2.9955 Acc: 0.0525

Epoch 5/25
----------
train Loss: 2.9995 Acc: 0.0534
val Loss: 2.9924 Acc: 0.0525

Epoch 6/25
----------
train Loss: 2.9964 Acc: 0.0534
val Loss: 2.9915 Acc: 0.0525

Epoch 7/25
----------
train Loss: 2.9967 Acc: 0.0482
val Loss: 2.9915 Acc: 0.0525

Epoch 8/25
----------
train Loss: 2.9921 Acc: 0.0490
val Loss: 2.9894 Acc: 0.0525

Epoch 9/25
----------
train Loss: 2.9901 Acc: 0.0539
val Loss: 2.9888 Acc: 0.0525

Epoch 10/25
----------
train Loss: 2.9930 Acc: 0.0411
val Loss: 2.9885 Acc: 0.0525

Epoch 11/25
----------
train Loss: 2.9896 Acc: 0.0513
val Loss: 2.9884 Acc: 0.0525

Epoch 12/25
----------
train Loss: 2.9902 Acc: 0.0542
val Loss: 2.9883 Acc: 0.0525

Epoch 13/25
----------


In [33]:
print('Testing')
model = models.vgg16()
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 20)
model = model.to(device)
model.load_state_dict(torch.load('/content/best_model_weight.pth'))

Testing


<All keys matched successfully>

In [34]:
y_true, y_pred = get_test_results(model, 'train')
print(f"Accuracy Score: {accuracy_score(y_true, y_pred)}\n")
print(f"Confusion Matrix\n")
print(np.array2string(confusion_matrix(y_true, y_pred)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Accuracy Score: 0.052369730295888976

Confusion Matrix

[[  0   0   0   0   0   0   0   0   0   0 200   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0 200   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0 200   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0 200   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0 200   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0 171   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0 200   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0 200   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0 200   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0 121   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0 200   0   0   0   0   0   0   0
    0   0]
 [  0   0

# Variation 5

In [35]:
# number of epochs : 20
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features

model.classifier[6] = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

#Training
model = train_model(model, criterion, optimizer, scheduler, num_epochs=20)

Epoch 1/20
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 1.5716 Acc: 0.5302
val Loss: 1.3721 Acc: 0.6008

Epoch 2/20
----------
train Loss: 1.1128 Acc: 0.6481
val Loss: 1.2404 Acc: 0.6176

Epoch 3/20
----------
train Loss: 0.9267 Acc: 0.7004
val Loss: 1.3344 Acc: 0.5882

Epoch 4/20
----------
train Loss: 0.7900 Acc: 0.7345
val Loss: 1.2667 Acc: 0.6092

Epoch 5/20
----------
train Loss: 0.6862 Acc: 0.7617
val Loss: 1.3381 Acc: 0.6197

Epoch 6/20
----------
train Loss: 0.6113 Acc: 0.7874
val Loss: 1.3137 Acc: 0.6197

Epoch 7/20
----------
train Loss: 0.5842 Acc: 0.7827
val Loss: 1.3227 Acc: 0.6176

Epoch 8/20
----------
train Loss: 0.4039 Acc: 0.8311
val Loss: 1.2897 Acc: 0.6492

Epoch 9/20
----------
train Loss: 0.3626 Acc: 0.8342
val Loss: 1.3200 Acc: 0.6555

Epoch 10/20
----------
train Loss: 0.3520 Acc: 0.8329
val Loss: 1.3391 Acc: 0.6513

Epoch 11/20
----------
train Loss: 0.3443 Acc: 0.8301
val Loss: 1.3534 Acc: 0.6450

Epoch 12/20
----------
train Loss: 0.3388 Acc: 0.8308
val Loss: 1.3590 Acc: 0.6429

Epoch 13/20
----------


In [36]:
print('Testing')
model = models.vgg16()
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 20)
model = model.to(device)
model.load_state_dict(torch.load('/content/best_model_weight.pth'))

Testing


<All keys matched successfully>

In [37]:
y_true, y_pred = get_test_results(model, 'train')
print(f"Accuracy Score: {accuracy_score(y_true, y_pred)}\n")
print(f"Confusion Matrix\n")
print(np.array2string(confusion_matrix(y_true, y_pred)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Accuracy Score: 0.8447237496726892

Confusion Matrix

[[196   0   0   1   0   0   3   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0 185   0   0   0   5   3   0   0   0   0   0   0   2   2   1   0   0
    1   1]
 [  0   0 193   1   0   0   1   0   1   0   0   1   0   0   1   0   1   0
    0   1]
 [  1   0   0 192   0   0   2   0   0   0   0   1   0   0   2   1   0   1
    0   0]
 [  0   7   0   0 143   0   2   3   2   0  14   3   1   0   3   2   1   8
    0  11]
 [  0   1   0   0   0 160   6   0   0   0   0   0   0   3   0   0   0   0
    1   0]
 [  3   6   0   2   1  12 162   0   0   0   0   2   0   5   5   0   0   0
    2   0]
 [  0   0   0   0   1   0   1 178   4   0   1   4   0   0   2   3   1   3
    0   2]
 [  0   1   0   0   8   0   1   2 107   0  22   3   0   0   4  13   0  11
    1  27]
 [  0   1   0   0   0   0   1   0   0 118   0   0   0   0   0   0   1   0
    0   0]
 [  0   0   0   0  16   0   0   0  14   0 142   1   0   0   2  15   0   5
    0   5]
 [  0   0  

# Variation 6

In [38]:
#Number of epochs : 30
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features

model.classifier[6] = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

#Training
model = train_model(model, criterion, optimizer, scheduler, num_epochs=30)

Epoch 1/30
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 1.5793 Acc: 0.5300
val Loss: 1.3691 Acc: 0.6008

Epoch 2/30
----------
train Loss: 1.1140 Acc: 0.6502
val Loss: 1.3105 Acc: 0.6239

Epoch 3/30
----------
train Loss: 0.9298 Acc: 0.6968
val Loss: 1.2875 Acc: 0.6239

Epoch 4/30
----------
train Loss: 0.7850 Acc: 0.7329
val Loss: 1.3330 Acc: 0.6197

Epoch 5/30
----------
train Loss: 0.6979 Acc: 0.7588
val Loss: 1.3451 Acc: 0.6471

Epoch 6/30
----------
train Loss: 0.6347 Acc: 0.7685
val Loss: 1.3163 Acc: 0.6387

Epoch 7/30
----------
train Loss: 0.5885 Acc: 0.7800
val Loss: 1.3080 Acc: 0.6576

Epoch 8/30
----------
train Loss: 0.3895 Acc: 0.8363
val Loss: 1.3457 Acc: 0.6639

Epoch 9/30
----------
train Loss: 0.3722 Acc: 0.8306
val Loss: 1.3630 Acc: 0.6576

Epoch 10/30
----------
train Loss: 0.3592 Acc: 0.8246
val Loss: 1.3707 Acc: 0.6660

Epoch 11/30
----------
train Loss: 0.3433 Acc: 0.8280
val Loss: 1.3614 Acc: 0.6618

Epoch 12/30
----------
train Loss: 0.3275 Acc: 0.8311
val Loss: 1.4314 Acc: 0.6639

Epoch 13/30
----------


In [39]:
print('Testing')
model = models.vgg16()
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 20)
model = model.to(device)
model.load_state_dict(torch.load('/content/best_model_weight.pth'))

Testing


<All keys matched successfully>

In [40]:
y_true, y_pred = get_test_results(model, 'train')
print(f"Accuracy Score: {accuracy_score(y_true, y_pred)}\n")
print(f"Confusion Matrix\n")
print(np.array2string(confusion_matrix(y_true, y_pred)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Accuracy Score: 0.8449855983241686

Confusion Matrix

[[194   0   0   1   0   0   5   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0 176   0   0   3   4   7   0   1   1   0   0   0   0   2   3   0   1
    1   1]
 [  0   0 197   0   0   0   0   0   1   0   0   1   0   0   0   0   1   0
    0   0]
 [  1   0   1 187   0   0   4   0   0   0   0   0   0   0   6   0   0   1
    0   0]
 [  0   3   0   0 121   0   2   4  10   0  24   5   1   0   7   2   1  10
    0  10]
 [  0   3   0   0   0 160   6   0   0   0   0   0   0   1   0   0   0   0
    1   0]
 [  1   2   1   0   1  13 165   1   2   1   0   4   0   4   4   0   0   0
    1   0]
 [  0   0   0   0   1   0   1 183   2   0   0   4   0   0   2   1   0   4
    0   2]
 [  0   0   0   0   2   0   0   4 118   0  25   2   0   0   5   7   0  20
    1  16]
 [  0   0   0   0   0   0   0   0   0 121   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   6   0   0   0  13   0 158   0   0   0   0   7   0  13
    0   3]
 [  0   0  